## Task 2 Method Chaining

### Method Chain

In [1]:
import pandas as pd
import numpy as np
def load_and_process(path):

# Method Chain 1, load data and drop and replace columns
    # Replace unknown data in Company Rating and Size Column 

    df = (pd.read_csv(path,sep=',')
          .drop(['Job Description',"Easy Apply","Competitors","Industry","Sector","Revenue","Type of ownership","Headquarters","Founded"],axis='columns') 
          .rename(columns = {"Rating" : "Company Rating","Size":"Company Size"})
         )

    df_replace = (pd.DataFrame(data=df)
           .loc[lambda x: x["Company Size"]=="-1"]
           .replace("-1","Unknown")
          )
    df.update(df_replace)
     

# Method Chain 2, Remove new line("\n") in Company name column
    # Remove (Glassdoor est.) in Salary Estimate column

    df2 = (pd.DataFrame(data=df["Company Name"])
          .replace(r'\n.*', '', regex=True)
         )
    df.update(df2)

    df2 = (pd.DataFrame(data=df["Salary Estimate"])
          .replace(r'\(.*', '', regex=True)
          .replace('[\$]', '', regex=True)
         )    
    df.update(df2)
        
    # Convert datatype in Salary Estimate column to int and Create new Min and Max column 

    new = df["Salary Estimate"].str.split("-", n = 1, expand = True)
    df["Minimum Salary"]= new[0]
    df["Maximum Salary"]= new[1]
    df.drop(columns =["Salary Estimate"],inplace = True)

    df2 = (pd.DataFrame(data=df["Minimum Salary"])
          .replace('K', '', regex=True)
         )
    df2["Minimum Salary"] = df2["Minimum Salary"].astype(int)
    df2=df2*1000
    df.update(df2)

    df2 = (pd.DataFrame(data=df["Maximum Salary"])
          .replace('K', '', regex=True)
         )
    df2["Maximum Salary"] = df2["Maximum Salary"].astype(int)
    df2=df2*1000
    df.update(df2)
    
    df["Average Salary"] = (df["Maximum Salary"]+df["Minimum Salary"])/2
    
    df["Maximum Salary"] = df["Maximum Salary"].astype(int)
    df["Minimum Salary"] = df["Minimum Salary"].astype(int)
    df["Average Salary"] = df["Average Salary"].astype(int)
    
# Cut down data size to only include Data Engineer, Software Engineer, and Big Data Engineer job title
     
    DataEng_data = df.loc[df["Job Title"]== "Data Engineer"] #469 rows
    SoftEng_data = df.loc[df["Job Title"]== "Software Engineer"] #93 rows
    BigDataEng_data = df.loc[df["Job Title"]== "Big Data Engineer"] #73 rows
    frames = [DataEng_data,SoftEng_data,BigDataEng_data]
    df = pd.concat(frames)
    df = df.reset_index()
    df = df.drop('index',axis='columns')

# Caterogized company size column
        
    df["Company Size"].loc[df["Company Size"] == "1 to 50 employees"] = "Small"
    df["Company Size"].loc[df["Company Size"] == "51 to 200 employees"] = "Small"
    df["Company Size"].loc[df["Company Size"] == "201 to 500 employees"] = "Medium"
    df["Company Size"].loc[df["Company Size"] == "501 to 1000 employees"] = "Large"
    df["Company Size"].loc[df["Company Size"] == "1001 to 5000 employees"] = "Large"
    df["Company Size"].loc[df["Company Size"] == "5001 to 10000 employees"] = "Large"
    df["Company Size"].loc[df["Company Size"] == "10000+ employees"] = "Large" 
    
# Lat and Long of each state for heatmap in dashboard  
    df["State"] = df["Location"].replace(r'^.*?,', '', regex=True)
    df["State"].loc[df["State"]==" NY"] = "40.730610,-73.935242"
    df["State"].loc[df["State"]==" IL"] = "41.881832,-87.623177"
    df["State"].loc[df["State"]==" CA"] = "36.778259,-119.417931"
    df["State"].loc[df["State"]==" TX"] = "31.000000,-100.000000"
    df["State"].loc[df["State"]==" AZ"] = "34.048927,-111.093735"
    df["State"].loc[df["State"]==" PA"] = "41.203323,-77.194527"
    df["State"].loc[df["State"]==" NJ"] = "39.833851,-74.871826"
    df["State"].loc[df["State"]==" DE"] = "39.000000,-75.500000"
    df["State"].loc[df["State"]==" FL"] = "27.994402,-81.760254"
    new2 = df["State"].str.split(",", n = 1, expand = True)
    df["Lat"]= new2[0]
    df["Long"]= new2[1]
    df.drop(columns =["State"],inplace = True)
    df["Lat"] = df["Lat"].astype(float)
    df["Long"] = df["Long"].astype(float)
    
    
    return (df)


In [2]:
df_clean = load_and_process('../data/raw/DataEngineer.csv')
df_clean.to_csv(r'../data/processed/clean_dataengineer.csv', index = False, header = True)

C:\Users\myste\miniconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
